In [27]:
import sys
import struct
import numpy as np
import nimrod_data_defs as ndd
import nimrod_funcs as nfc
import glob
import os

In [2]:
year = 2018

In [15]:
# os.chdir("../../../datadir/NimRod/2018")

In [28]:
def read_nimrod_data(input_file):
    print("Input file is", input_file)

    # Open the file for reading and check the initial header size
    file_id = open(input_file, "rb")
    record_length, = struct.unpack(">l", file_id.read(4))
    if (record_length != 512):
        raise RuntimeError("Incorrect record length", record_length)

    # Read the four main headers from the file
    print(" -   Reading header")
    int_hdr_bytes = np.fromfile(file_id,
                                dtype=ndd.main_int_header_fmt,
                                count=1)
    real_hdr_bytes = np.fromfile(file_id,
                                 dtype=ndd.main_real_header_fmt,
                                 count=1)
    char_hdr_bytes = np.fromfile(file_id,
                                 dtype=ndd.main_char_header_fmt,
                                 count=1)
    data_hdr_bytes = np.fromfile(file_id,
                                 dtype=ndd.data_header_fmt,
                                 count=1)
    record_length, = struct.unpack(">l", file_id.read(4))
    if (record_length != 512):
        raise RuntimeError("Incorrect record length", record_length)

    # Convert the four headers into one large header dictionary
    nimrod_output = {
        "I_Hdr": nfc.recarray2dict(int_hdr_bytes),
        "R_Hdr": nfc.recarray2dict(real_hdr_bytes),
        "C_Hdr": nfc.recarray2dict(char_hdr_bytes),
        "D_Hdr": nfc.recarray2dict(data_hdr_bytes)
        }
    print(" -   Header successfully read")

    # Compute size of the actual data array using header info
    data_array_size = (nimrod_output["I_Hdr"]['NumRows'][0].astype(np.int32) *
                       nimrod_output["I_Hdr"]['NumCols'][0].astype(np.int32))

    # Check the data record size is consistent with the array size
    record_length, = struct.unpack(">l", file_id.read(4))
    if (record_length != data_array_size * 2):
        raise RuntimeError("Incorrect record length. Got", record_length,
                           "expected", data_array_size * 2)

    # Create the data array format, then read data from file
    data_fmt = nfc.create_data_dtype(nimrod_output["I_Hdr"]['NumRows'][0],
                                     nimrod_output["I_Hdr"]['NumCols'][0])
    data_array = np.fromfile(file_id, dtype=data_fmt, count=1)

    # Radar data stores values multiplied by 32.
    nimrod_output['Data'] = np.squeeze(data_array['Data']) / 32.

    # Done reading data
    file_id.close()
    print(" -   Data successfully read")

    return nimrod_output


In [49]:
if not os.path.isdir(f"../../../datadir/NimRod_Processed/{year}/"):
    os.makedirs(f"../../../datadir/NimRod_Processed/{year}/")
    print ('Data directory created')
os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/NimRod/{year}/Newfolder")

In [50]:
pwd

'/nfs/a319/gy17m2a/PhD/datadir/NimRod/2015/Newfolder'

In [51]:
for input_filename in os.listdir():
    print(input_filename)
    output_filename = input_filename.replace("NimRod", 'NimRod_Processed')
    output_filename = output_filename.replace(".dat", '.tiff')
#     # Call the main function
    nimrod = read_nimrod_data(input_filename)
    nfc.create_gdal_output(nimrod, output_filename)    

metoffice-c-band-rain-radar_uk_201506010000_1km-composite.dat
Input file is metoffice-c-band-rain-radar_uk_201506010000_1km-composite.dat
 -   Reading header
 -   Header successfully read
 -   Data successfully read


In [33]:
nimrod = read_nimrod_data("metoffice-c-band-rain-radar_uk_200509021525_1km-composite.dat")

Input file is metoffice-c-band-rain-radar_uk_200509021525_1km-composite.dat
 -   Reading header
 -   Header successfully read
 -   Data successfully read


In [12]:
output_filename = input_filename.replace("NimRod", 'NimRod_Processing')
output_filename = output_filename.replace(".dat", '.tiff')
output_filename

'../../../../datadir/NimRod_Processing/2005/metoffice-c-band-rain-radar_uk_200512272055_1km-composite.tiff'